### **一、Drain3**
#### 1. 算法简介
&emsp;&emsp;Drain是2017年提出的在线日志解析方法，Drain3将Drain进行了改造使其能过够再Python3下使用并使用与实际的生产环境。Drain3的核心思想与Drain一致，都是基于“同一个日志模板产生的日志经过分词得到的词的个数是相同的”这一假设，在词假设前提下，建立一个由日志长度、前缀单词和日志模板组成的Parse Tree。
#### 2. 使用场景
&emsp;&emsp;本算法适用于对日志进行<font color=red>实时解析和分析</font>，准确率和效率较高。
#### 3. 算法原理
![Excalidraw Image](../pictures/drain3.png)<br>
&emsp;&emsp;对于输入的日志首先会使用正则表达式对常用变量进行替换，然后再进行分词，根据分词的长度将日志分到不同的子节点中，然后再根据第一个分词分到不同的groups中。分到对应的日志groups桶中后，将会计算日志与每个group的模式之间的相似度，并取最大值与阈值进行比较，如果相似度大于阈值则将日志归入相应group并将模式中与日志不同的部分用*替代，返回该模式；如果相似度小于阈值则创建一个新的group，group的模式为日志本身并将其返回。以下是Drain3对日志进行模式发现的example:<br>
![Excalidraw Image](../pictures/drain3_example.png)<br>
**论文原文链接**：<https://jiemingzhu.github.io/pub/pjhe_icws2017.pdf>
#### 4. 运行示例
(1) 对本地的.log文件中的日志进行模式提取，将训练得到的模型保存在指定目录，并返回模式发现的结果

In [4]:
import os
import pickle
from drain3 import TemplateMiner
from drain3.template_miner_config import TemplateMinerConfig

# 从log文件目录读取文件
log_file = '../test_data/mixdata.log'
with open(log_file) as f:
    logs = f.readlines()

# 进行drain3算法相关配置（可在同目录下drain3.ini文件进行修改）
config = TemplateMinerConfig()
config.load('drain3.ini')
# config.load(os.path.join(os.path.dirname(__file__), 'drain3.ini'))
config.profiling_enabled = False
template_miner = TemplateMiner(config=config)

# 模式发现的结果，template，size
results = {}

# 对每条日志进行模板提取
for log in logs:
    log = log.rstrip()
    log = log.partition(': ')[2]
    result = template_miner.add_log_message(log)
    cluster_id = result['cluster_id']
    results[cluster_id] = {
        'template': result['template_mined'],
        'size': result['cluster_size']
    }

# 将训练好的template_miner模型保存到model_path
model_dir = '../models'
model_path = os.path.join(model_dir, 'model2')
with open(model_path, 'wb') as file:
    pickle.dump(template_miner, file)

# 模式发现的结果
results = list(results.values())
print(f'共发现{len(results)}种日志模式：')
print(results)

共发现907种日志模式：
[{'template': 'authentication failure; logname= uid=<:*:> euid=<:*:> <:*:> ruser= <:*:>', 'size': 118}, {'template': 'check pass; user unknown', 'size': 117}, {'template': 'authentication failure; logname= uid=<:*:> euid=<:*:> tty=NODEVssh ruser= <:*:> <:*:>', 'size': 372}, {'template': 'session opened for user <:*:> by <:*:>', 'size': 123}, {'template': 'session closed for user <:*:>', 'size': 123}, {'template': 'ALERT exited abnormally with [<:*:>]', 'size': 43}, {'template': 'IOThunderboltSwitch<<:*:>>(<:*:>)::listenerCallback - Thunderbolt HPD packet for route = <:*:> port = <:*:> unplug = <:*:>', 'size': 36}, {'template': 'Thermal pressure state: <:*:> Memory pressure state: <:*:>', 'size': 39}, {'template': 'FA||Url||taskID[<:*:>] dealloc', 'size': 17}, {'template': 'ARPT: <:*:>.<:*:>: <:*:> <:*:> <:*:>', 'size': 46}, {'template': 'mDNS <:*:> Frequent transitions for interface <:*:> <:*:>', 'size': 9}, {'template': "ARPT: <:*:>.<:*:>: IOPMPowerSource Information: <:*

(2) 从前端获取一批日志进行模式提取，可以选择已训练好的模型进行模式发现，也可以直接进行模式发现（参考上一示例），返回模式发现的结果

In [7]:
# 前端获取的一批日志
logs = ['081109 203615 148 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_38865049064139660 terminating',
        '081109 205056 710 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_5017373558217225674 terminating',
        '081109 205035 28 INFO dfs.FSNamesystem: BLOCK* NameSystem.allocateBlock: /user/root/rand/_temporary/_task_200811092030_0001_m_000590_0/part-00590. blk_-1727475099218615100',
        '081109 210022 1110 INFO dfs.DataNode$PacketResponder: Received block blk_-5974833545991408899 of size 67108864 from /10.251.31.180',
        '081109 205931 13 INFO dfs.DataBlockScanner: Verification succeeded for blk_-4980916519894289629',
        '081109 210551 32 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.250.6.191:50010 is added to blk_673825774073966710 size 67108864',
        '081109 212220 1946 INFO dfs.DataNode$DataXceiver: Receiving block blk_-774267833966018354 src: /10.251.38.53:51057 dest: /10.251.38.53:50010',
        ]

# 从指定目录获取已训练好的模型
model_path = '../models/model1'
with open(model_path, "rb") as file:
    predict_model = pickle.load(file)

# 模式发现的结果，template，size
results = {}

# 对每条日志进行模板提取
for log in logs:
    log = log.rstrip()
    log = log.partition(': ')[2]
    cluster = predict_model.match(log)
    # match template
    if cluster is None:
        if 0 not in results:
            results[0] = {'template': 'No match found', 'size': 0}
        results[0]['size'] += 1
    else:
        cluster_id = cluster.cluster_id
        if cluster_id not in results:
            results[cluster_id] = {'template': cluster.get_template(), 'size': 0}
        results[cluster_id]['size'] += 1

# 模式发现的结果
results = list(results.values())
print(f'共发现{len(results)}种日志模式：')
print(results)

共发现6种日志模式：
[{'template': 'PacketResponder <:*:> for block blk <:*:> terminating', 'size': 2}, {'template': 'BLOCK* NameSystem.allocateBlock: <:*:> temporary/ task <:*:> <:*:> <:*:> <:*:> <:*:>/part-<:*:>. blk <:*:>', 'size': 1}, {'template': 'Received block blk <:*:> of size <:*:> from /<:*:>', 'size': 1}, {'template': 'Verification succeeded for blk <:*:>', 'size': 1}, {'template': 'BLOCK* NameSystem.addStoredBlock: blockMap updated: <:*:>:<:*:> is added to blk <:*:> size <:*:>', 'size': 1}, {'template': 'Receiving block blk <:*:> src: /<:*:>:<:*:> dest: /<:*:>:<:*:>', 'size': 1}]


(3) 此外，Drain3还实现了多种持久化，可以在初始化template_miner模型的时候根据持久化的内容进行初始化

In [13]:
# 可指定持久化类型
persistence_type = 'FILE'

if persistence_type == "KAFKA":
    from drain3.kafka_persistence import KafkaPersistence

    persistence = KafkaPersistence("drain3_state", bootstrap_servers="localhost:7890")
# 9092

elif persistence_type == "FILE":
    from drain3.file_persistence import FilePersistence

    persistence = FilePersistence("drain3_state.bin")

elif persistence_type == "REDIS":
    from drain3.redis_persistence import RedisPersistence

    persistence = RedisPersistence(redis_host='',
                                   redis_port=25061,
                                   redis_db=0,
                                   redis_pass='',
                                   is_ssl=True,
                                   redis_key="drain3_state_key")
else:
    persistence = None

# 进行drain3相关配置
config = TemplateMinerConfig()
config.load('drain3.ini')
config.profiling_enabled = False

# 根据持久化内容对模型进行初始化配置
template_miner = TemplateMiner(persistence, config)